#Training  a Random Forest to Classify the review of a book using Amazon books Review Dataset by reading at as a Pyspark Dataframe.


##Dataset URL-https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews

##Installing pyspark

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c9cdf9c217b2b1e07cd25ac59e43b63bc998b8cbee7217022f220d4afd892508
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"navneet14","key":"a8d8b713c16b1d51f09f2ee56e2b3d59"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!mv /content/kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 99% 1.06G/1.06G [00:18<00:00, 49.7MB/s]
100% 1.06G/1.06G [00:18<00:00, 62.2MB/s]


In [ ]:
!unzip /content/amazon-books-reviews.zip

Archive:  /content/amazon-books-reviews.zip
  inflating: Books_rating.csv        
  inflating: books_data.csv          


#Importing neccessary libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, concat, lit
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import RandomForestClassifier


##Initialize Spark session

In [ ]:
spark = SparkSession.builder.appName("AmazonBookReviewClassification").getOrCreate()

In [ ]:
# Loading  the dataset
df = spark.read.csv("/content/Books_rating.csv", header=True)

#Let's take a look at the shape of our spark dataframe

In [ ]:
df.count()

3000000

In [ ]:
len(df.columns)

10

In [ ]:
df.columns

['Id',
 'Title',
 'Price',
 'User_id',
 'profileName',
 'review/helpfulness',
 'review/score',
 'review/time',
 'review/summary',
 'review/text']

#Now let's take a preliminary look at our dataset

In [ ]:
df.show()

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| NULL| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...| NULL|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...| NULL|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|"If people become...|
|0826414346|Dr. Seuss: Ameri

#As we can see our dataset consist of about 30,000,00 rows which is very huge to process so we will take 40% of the total data about 12,000,00 rows

In [ ]:
 df=df.sample(withReplacement=False,fraction=0.4,seed=42)

#Number of rows in new DataFrame

In [ ]:
df.count()

1202902

#By looking at our data set we can clearly see columns which are not of our use so we will make a seperate dataframe of required columns:

#"review/score", "review/summary", "review/text"

In [ ]:
req_df = df.select("review/score", "review/summary", "review/text")

In [ ]:
req_df.columns

['review/score', 'review/summary', 'review/text']

In [ ]:
req_df.show()

+------------+--------------------+--------------------+
|review/score|      review/summary|         review/text|
+------------+--------------------+--------------------+
|         4.0|Phlip Nel gives s...|Theodore Seuss Ge...|
|         5.0|Academia At It's ...|"When I recieved ...|
|         5.0|Not Just for Past...|I just finished r...|
|         1.0|        Buyer beware|"This is a self-p...|
|         5.0|      Bravo Veronica|I read the review...|
|         5.0|Whispers of the W...|This was a easy t...|
|         4.0|Errors, but great...|I was a dissapoin...|
|         1.0|          The Worst!|A complete waste ...|
|         5.0|The best book I h...|I happen to love ...|
|         5.0|Couldn't put this...|Normally I'm not ...|
|         1.0|Awful beyond belief!|"I feel I have to...|
|         1.0|You've got to be ...|"It has been year...|
|         5.0|Early Christian d...|Everett Ferguson ...|
|         4.0|      Christ is Lord|This is a very us...|
|         5.0| A walk in the wo

#In our dataset we do not have any Y feature so we will create Y according to the review/score feature in our dataset

In [ ]:
# Labeling  the review score
req_df = req_df.withColumn("Y", when(col("review/score") >= 3, 1).otherwise(0))

In [ ]:
req_df.show(5)

+------------+--------------------+--------------------+---+
|review/score|      review/summary|         review/text|  Y|
+------------+--------------------+--------------------+---+
|         4.0|Phlip Nel gives s...|Theodore Seuss Ge...|  1|
|         5.0|Academia At It's ...|"When I recieved ...|  1|
|         5.0|Not Just for Past...|I just finished r...|  1|
|         1.0|        Buyer beware|"This is a self-p...|  0|
|         5.0|      Bravo Veronica|I read the review...|  1|
+------------+--------------------+--------------------+---+
only showing top 5 rows



#Dump the original review/score column to maintain readability or it is no longer in our use

In [ ]:
# Drop the original score column
req_df = req_df.drop("review/score")

In [ ]:
req_df.show(5)

+--------------------+--------------------+---+
|      review/summary|         review/text|  Y|
+--------------------+--------------------+---+
|Phlip Nel gives s...|Theodore Seuss Ge...|  1|
|Academia At It's ...|"When I recieved ...|  1|
|Not Just for Past...|I just finished r...|  1|
|        Buyer beware|"This is a self-p...|  0|
|      Bravo Veronica|I read the review...|  1|
+--------------------+--------------------+---+
only showing top 5 rows



#Now we will be combining the summary and review/summary features as they are Moreover same and together will give more idea about the sentiments

In [ ]:
# Combine summary and text columns
req_df = req_df.withColumn("summary", concat(col("review/summary"), lit(" "), col("review/text")))

In [ ]:
req_df.show(5)

+--------------------+--------------------+---+--------------------+
|      review/summary|         review/text|  Y|             summary|
+--------------------+--------------------+---+--------------------+
|Phlip Nel gives s...|Theodore Seuss Ge...|  1|Phlip Nel gives s...|
|Academia At It's ...|"When I recieved ...|  1|Academia At It's ...|
|Not Just for Past...|I just finished r...|  1|Not Just for Past...|
|        Buyer beware|"This is a self-p...|  0|Buyer beware "Thi...|
|      Bravo Veronica|I read the review...|  1|Bravo Veronica I ...|
+--------------------+--------------------+---+--------------------+
only showing top 5 rows



#Dumping the "review/summary" and "review/text" features as they are no longer of our use

In [ ]:
# Drop the original summary and text columns
columns_to_drop = ["review/summary", "review/text"]
req_df = req_df.drop(*columns_to_drop)

In [ ]:
req_df.show(5)

+---+--------------------+
|  Y|             summary|
+---+--------------------+
|  1|Phlip Nel gives s...|
|  1|Academia At It's ...|
|  1|Not Just for Past...|
|  0|Buyer beware "Thi...|
|  1|Bravo Veronica I ...|
+---+--------------------+
only showing top 5 rows



In [ ]:
null_values_df = req_df.filter(
    col("summary").isNull()
)

In [ ]:
null_values_df.count()

32

#As we can see there are 32 null values in summary it can cause errors in our upcoming operations like tokenization and etc.

# As it is not a big number let's fill all those with the empty string

In [ ]:
# Replace null values with empty string
req_df = req_df.na.fill("")

#let's check if above code works or not

In [ ]:
null_values_df = req_df.filter(
    col("summary").isNull()
)
null_values_df.count()

0

# Now we will split the text into individual words or tokens, which is a common preprocessing step in natural language processing (NLP) tasks such as text classification, sentiment analysis, etc.

#For this we will use ```tokenizer``` method available in pyspark and will appply the result in our dataframe using transform.

In [ ]:
# Tokenize the summary
tokenizer = Tokenizer(inputCol="summary", outputCol="words")
tokenized_df = tokenizer.transform(req_df)


In [ ]:
tokenized_df.show()

+---+--------------------+--------------------+
|  Y|             summary|               words|
+---+--------------------+--------------------+
|  1|Phlip Nel gives s...|[phlip, nel, give...|
|  1|Academia At It's ...|[academia, at, it...|
|  1|Not Just for Past...|[not, just, for, ...|
|  0|Buyer beware "Thi...|[buyer, beware, "...|
|  1|Bravo Veronica I ...|[bravo, veronica,...|
|  1|Whispers of the W...|[whispers, of, th...|
|  1|Errors, but great...|[errors,, but, gr...|
|  0|The Worst! A comp...|[the, worst!, a, ...|
|  1|The best book I h...|[the, best, book,...|
|  1|Couldn't put this...|[couldn't, put, t...|
|  0|Awful beyond beli...|[awful, beyond, b...|
|  0|You've got to be ...|[you've, got, to,...|
|  1|Early Christian d...|[early, christian...|
|  1|Christ is Lord Th...|[christ, is, lord...|
|  1|A walk in the woo...|[a, walk, in, the...|
|  1|A fascinating ins...|[a, fascinating, ...|
|  1|A must have "Sorr...|[a, must, have, "...|
|  1|Practical Guide t...|[practical, gu

#Now we will remove common words such as "the", "is", "and", "but", etc., which are often filtered out because they do not add much meaning to the text these are called stopwords.

#To perfrom this we had ```StopWordsRemover```method available in pyspark



In [ ]:
# Remove stopwords
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
filtered_df = remover.transform(tokenized_df)

In [ ]:
filtered_df.show()

+---+--------------------+--------------------+--------------------+
|  Y|             summary|               words|            filtered|
+---+--------------------+--------------------+--------------------+
|  1|Phlip Nel gives s...|[phlip, nel, give...|[phlip, nel, give...|
|  1|Academia At It's ...|[academia, at, it...|[academia, best, ...|
|  1|Not Just for Past...|[not, just, for, ...|[pastors!, finish...|
|  0|Buyer beware "Thi...|[buyer, beware, "...|[buyer, beware, "...|
|  1|Bravo Veronica I ...|[bravo, veronica,...|[bravo, veronica,...|
|  1|Whispers of the W...|[whispers, of, th...|[whispers, wicked...|
|  1|Errors, but great...|[errors,, but, gr...|[errors,, great, ...|
|  0|The Worst! A comp...|[the, worst!, a, ...|[worst!, complete...|
|  1|The best book I h...|[the, best, book,...|[best, book, ever...|
|  1|Couldn't put this...|[couldn't, put, t...|[put, book, down!...|
|  0|Awful beyond beli...|[awful, beyond, b...|[awful, beyond, b...|
|  0|You've got to be ...|[you've,

#Now we have our required features ```filtered and Y``` we will no longer need other features ```summary and words``` .

#so, we will drop them

In [ ]:
filtered_df = filtered_df.drop("summary", "words")


In [ ]:
filtered_df.show(5)

+---+--------------------+
|  Y|            filtered|
+---+--------------------+
|  1|[phlip, nel, give...|
|  1|[academia, best, ...|
|  1|[pastors!, finish...|
|  0|[buyer, beware, "...|
|  1|[bravo, veronica,...|
+---+--------------------+
only showing top 5 rows



#HashingTF is a practical choice for feature extraction and text vectorization in PySpark, especially when dealing with large-scale text datasets.

#In this example SparseVector, the format

#(10000,[72,199,543,891,1000,2500,3500,5000,7500]

#can be interpreted as follows:

#Represents the indices of non-zero elements in the vector. Each index corresponds to a feature that has a non-zero value in this document's feature vector.

#where as 10000 indicates the total number of features (specified by numFeatures)

In [ ]:
# Apply HashingTF
hashing_tf = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
featurized_df = hashing_tf.transform(filtered_df)


In [ ]:
featurized_df.show()

+---+--------------------+--------------------+
|  Y|            filtered|         rawFeatures|
+---+--------------------+--------------------+
|  1|[phlip, nel, give...|(10000,[7,177,190...|
|  1|[academia, best, ...|(10000,[393,507,7...|
|  1|[pastors!, finish...|(10000,[7,44,86,2...|
|  0|[buyer, beware, "...|(10000,[145,281,3...|
|  1|[bravo, veronica,...|(10000,[55,393,50...|
|  1|[whispers, wicked...|(10000,[353,393,6...|
|  1|[errors,, great, ...|(10000,[80,353,39...|
|  0|[worst!, complete...|(10000,[953,992,1...|
|  1|[best, book, ever...|(10000,[157,225,3...|
|  1|[put, book, down!...|(10000,[393,524,1...|
|  0|[awful, beyond, b...|(10000,[25,42,298...|
|  0|[got, kidding!, "...|(10000,[1,281,566...|
|  1|[early, christian...|(10000,[393,815,8...|
|  1|[christ, lord, us...|(10000,[666,1119,...|
|  1|[walk, woods, old...|(10000,[30,561,68...|
|  1|[fascinating, ins...|(10000,[327,393,4...|
|  1|[must, "sorry, mi...|(10000,[440,635,7...|
|  1|[practical, guide...|(10000,[393,44

#Inverse Document Frequency (IDF) is a concept commonly used in natural language processing and information retrieval to measure the importance of a term in a collection of documents. It helps in identifying terms that are rare and therefore more informative about the content of a document.

#$\textbf{Mathematical Expression:}$

#The IDF of a term \( t \) is calculated using the formula:

#$ \text{IDF}(t) = \log \left( \frac{N}{df(t)} \right) $

#Where:

  - $ N$ is the total number of documents in the collection.
  - $ df(t)$ is the number of documents containing term t.




- IDF gives higher weights to terms that are rare in the document collection but occur frequently in specific documents.


- Terms that are common across many documents (e.g., stop words like "the," "and," etc.) have lower IDF values.


#IDF provides a form of ```normalization``` by scaling down the importance of terms based on their frequency across the entire document collection. This normalization is beneficial for various text processing tasks, including text classification, clustering, and information retrieval.

#Let's understand the working of ```idf``` more deeply


Imagine you have a bunch of documents, and you want to figure out which words are important in these documents. IDF helps with that. Now, here's what

```idf_model = idf.fit(featurized_df)``` means:

#What's IDF doing?

- IDF looks at all your documents and checks how often each word appears across all the documents. Words that appear a lot (like "the" or "and") get less importance because they're common. But words that show up rarely but are still meaningful get more importance.

# What's fit doing?

- The fit part is like teaching IDF about your specific documents. It looks at the words in your documents and learns which words are common and which are rare. It then calculates how important each word is based on this learning.


- After using fit, you get an ```idf_model```. This model is like a trained IDF expert for your documents. It knows which words are important in your documents based on their rarity and relevance across your whole document collection.


##Once you have ```idf_model```, you can use it to figure out the importance of words in new documents too. So, it's not just for the documents you trained it on; it's ready to help with any new text you want to analyze.

In [ ]:
# Apply IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf_model = idf.fit(featurized_df)

In [ ]:
after_idf= idf_model.transform(featurized_df)


In [ ]:
after_idf.show()

+---+--------------------+--------------------+--------------------+
|  Y|            filtered|         rawFeatures|            features|
+---+--------------------+--------------------+--------------------+
|  1|[phlip, nel, give...|(10000,[7,177,190...|(10000,[7,177,190...|
|  1|[academia, best, ...|(10000,[393,507,7...|(10000,[393,507,7...|
|  1|[pastors!, finish...|(10000,[7,44,86,2...|(10000,[7,44,86,2...|
|  0|[buyer, beware, "...|(10000,[145,281,3...|(10000,[145,281,3...|
|  1|[bravo, veronica,...|(10000,[55,393,50...|(10000,[55,393,50...|
|  1|[whispers, wicked...|(10000,[353,393,6...|(10000,[353,393,6...|
|  1|[errors,, great, ...|(10000,[80,353,39...|(10000,[80,353,39...|
|  0|[worst!, complete...|(10000,[953,992,1...|(10000,[953,992,1...|
|  1|[best, book, ever...|(10000,[157,225,3...|(10000,[157,225,3...|
|  1|[put, book, down!...|(10000,[393,524,1...|(10000,[393,524,1...|
|  0|[awful, beyond, b...|(10000,[25,42,298...|(10000,[25,42,298...|
|  0|[got, kidding!, "...|(10000,[

#Let's split our data in training and testing 70% training 30% testing

In [ ]:
# Split the data into training and testing sets
train_df, test_df = after_idf.randomSplit([0.7, 0.3], seed=45)

#Random Forest Overview
A random forest is an ensemble learning method that consists of a large number of individual decision trees. Each tree is trained on a subset of the training data (sampled with replacement, also known as bootstrap sampling) and a random subset of features. The final prediction for a given input is based on aggregating the predictions from all individual trees.
##Training Phase:
- During training, each decision tree is built using a random subset of the data and a random subset of features.
- Each tree learns to classify the input based on the training data it sees.

##Prediction Phase:

- When a new data point (a feature vector) is presented to the trained random forest model, it is passed through each of the individual decision trees.
- Each decision tree makes its own classification decision for the data point.

##Voting:
- The ```rawPrediction``` is a vector that represents the count of votes from all the trees for each class.

- For a binary classification problem (classes 0 and 1), the rawPrediction vector might look like ```[number_of_votes_for_class_0,number_of_votes_for_class_1]```.



#```rf = RandomForestClassifier(labelCol="Y", featuresCol="features", numTrees=100)```

##```RandomForestClassifier``` takes arguments ```labelCol,featuresCol and numTrees ```  

- ```labelCol:``` The name of the column in your DataFrame that contains the labels or target variable you want to predict.

- ```featuresCol:``` The name of the column in your DataFrame that contains the features or input variables used for making predictions.

- ```numTrees:``` The number of trees in the forest. More trees generally improve the model performance but increase computational time.


In [ ]:
# Train the Random Forest classifier
rf = RandomForestClassifier(labelCol="Y", featuresCol="features", numTrees=100)
rf_model = rf.fit(train_df)

In [ ]:
# Make predictions and evaluate the model
predictions = rf_model.transform(test_df)


In [ ]:
predictions.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  Y|            filtered|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0|                  []|(10000,[3372],[1.0])|(10000,[3372],[5....|[12.3062121326274...|[0.12306212132627...|       1.0|
|  0|[, hist...", 414/...|(10000,[3372,5552...|(10000,[3372,5552...|[12.3062121326274...|[0.12306212132627...|       1.0|
|  0|["""eragon, fathe...|(10000,[15,94,280...|(10000,[15,94,280...|[13.1266468235252...|[0.13126646823525...|       1.0|
|  0|["""honey, , writ...|(10000,[1307,2063...|(10000,[1307,2063...|[12.4594075516583...|[0.12459407551658...|       1.0|
|  0|["""revelation"",...|(10000,[177,393,4...|(10000,[177,393,4...|[13.4124561464641...|[0.13412456146464...|       1.0|
|  0|["""spot, heresy"..

In [ ]:

predictions.select("rawPrediction", "probability","prediction").show(truncate=False)

+--------------------------------------+----------------------------------------+----------+
|rawPrediction                         |probability                             |prediction|
+--------------------------------------+----------------------------------------+----------+
|[12.306212132627449,87.69378786737258]|[0.12306212132627445,0.8769378786737255]|1.0       |
|[12.306212132627449,87.69378786737258]|[0.12306212132627445,0.8769378786737255]|1.0       |
|[13.126646823525219,86.8733531764748] |[0.13126646823525218,0.8687335317647478]|1.0       |
|[12.459407551658385,87.54059244834164]|[0.12459407551658382,0.8754059244834161]|1.0       |
|[13.412456146464121,86.5875438535359] |[0.1341245614646412,0.8658754385353588] |1.0       |
|[12.54982775279154,87.45017224720851] |[0.12549827752791531,0.8745017224720847]|1.0       |
|[12.337870038089891,87.66212996191014]|[0.12337870038089888,0.8766212996191012]|1.0       |
|[13.28151224557976,86.71848775442025] |[0.13281512245579757,0.8671848

In [ ]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#Calculating Performance Metrics of our model

In [ ]:
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="Y", predictionCol="prediction")

In [ ]:

accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


Accuracy: 0.8770776179787735
F1 Score: 0.8196412770481593
Precision: 0.7692651479593194
Recall: 0.8770776179787735
